#  图像分类模型通道剪裁-快速开始

该教程以图像分类模型MobileNetV1为例，说明如何快速使用[PaddleSlim的卷积通道剪裁接口]()。
该示例包含以下步骤：

1. 导入依赖
2. 构建模型
3. 剪裁
4. 训练剪裁后的模型

以下章节依次次介绍每个步骤的内容。

## 1. 导入依赖

PaddleSlim依赖Paddle1.7版本，请确认已正确安装Paddle，然后按以下方式导入Paddle和PaddleSlim:

In [22]:
import paddle
import paddle.fluid as fluid
import paddleslim as slim

## 2. 构建网络

该章节构造一个用于对MNIST数据进行分类的分类模型，选用`MobileNetV1`，并将输入大小设置为`[1, 28, 28]`，输出类别数为10。
为了方便展示示例，我们在`paddleslim.models`下预定义了用于构建分类模型的方法，执行以下代码构建分类模型：

In [23]:
exe, train_program, val_program, inputs, outputs = slim.models.image_classification("MobileNet", [1, 28, 28], 10, use_gpu=False)

>注意：paddleslim.models下的API并非PaddleSlim常规API，是为了简化示例而封装预定义的一系列方法，比如：模型结构的定义、Program的构建等。

## 3. 剪裁卷积层通道

### 3.1 计算剪裁之前的FLOPs

In [24]:
FLOPs = slim.analysis.flops(train_program)
print("FLOPs: {}".format(FLOPs))

FLOPs: 10907072.0


### 3.2 剪裁

我们这里对参数名为`conv2_1_sep_weights`和`conv2_2_sep_weights`的卷积层进行剪裁，分别剪掉20%和30%的通道数。
代码如下所示：

In [25]:
pruner = slim.prune.Pruner()
pruned_program, _, _ = pruner.prune(
        train_program,
        fluid.global_scope(),
        params=["conv2_1_sep_weights", "conv2_2_sep_weights"],
        ratios=[0.33] * 2,
        place=fluid.CPUPlace())

以上操作会修改`train_program`中对应卷积层参数的定义，同时对`fluid.global_scope()`中存储的参数数组进行裁剪。

### 3.3 计算剪裁之后的FLOPs

In [26]:
FLOPs = paddleslim.analysis.flops(train_program)
print("FLOPs: {}".format(FLOPs))

FLOPs: 10907072.0


## 4. 训练剪裁后的模型

### 4.1 定义输入数据

为了快速执行该示例，我们选取简单的MNIST数据，Paddle框架的`paddle.dataset.mnist`包定义了MNIST数据的下载和读取。
代码如下：

In [27]:
import paddle.dataset.mnist as reader
train_reader = paddle.io.batch(
        reader.train(), batch_size=128, drop_last=True)
train_feeder = fluid.DataFeeder(inputs, fluid.CPUPlace())

### 4.2 执行训练
以下代码执行了一个`epoch`的训练：

In [21]:
for data in train_reader():
    acc1, acc5, loss = exe.run(pruned_program, feed=train_feeder.feed(data), fetch_list=outputs)
    print(acc1, acc5, loss)

[0.1484375] [0.4921875] [2.6727316]
[0.125] [0.546875] [2.6547904]
[0.125] [0.5546875] [2.795205]
[0.1171875] [0.578125] [2.8561475]
[0.1875] [0.59375] [2.470603]
[0.1796875] [0.578125] [2.8031898]
[0.1484375] [0.6015625] [2.7530417]
[0.1953125] [0.640625] [2.711596]
[0.125] [0.59375] [2.8637898]
[0.1796875] [0.53125] [2.9473038]
[0.25] [0.671875] [2.3943179]
[0.25] [0.6953125] [2.632146]
[0.2578125] [0.7265625] [2.723265]
[0.359375] [0.765625] [2.4263484]
[0.3828125] [0.8203125] [2.226284]
[0.421875] [0.8203125] [1.8042578]
[0.4765625] [0.890625] [1.6841211]
[0.53125] [0.8671875] [2.1971617]
[0.5546875] [0.8984375] [1.5361531]
[0.53125] [0.890625] [1.7211896]
[0.5078125] [0.8984375] [1.6586945]
[0.53125] [0.9140625] [1.8980236]
[0.546875] [0.9453125] [1.5279069]
[0.5234375] [0.8828125] [1.7356458]
[0.6015625] [0.9765625] [1.0375824]
[0.5546875] [0.921875] [1.639497]
[0.6015625] [0.9375] [1.5469061]
[0.578125] [0.96875] [1.3573356]
[0.65625] [0.9453125] [1.3787829]
[0.640625] [0.976562

KeyboardInterrupt: 